In [1]:
import numpy as np
import rebound
import matplotlib.pyplot as plt
from numpy.linalg import norm
plt.style.use('dynamics_site')


In [2]:
sim = rebound.Simulation()
r12 = 1.0 #initial spacing
m1=1.0
m2 = 0.1
y0 = 1.5 #0.5
x0 = 0 #0.5
vx = -1.1 #-0.8

sim.add(m=1)
sim.add(m=0.1,a=r12)
sim.move_to_com()
sim.add(m=0,x=x0,vx=vx,y=y0)

In [3]:
sim.dt = 0.001 * sim.particles[1].P  # 5% of secondary's period to start
Nout = 10000           # number of points to display
tmax = 200         # let the simulation run for 80 years

In [4]:
x = np.zeros((sim.N,Nout))
y = np.zeros((sim.N,Nout))
z = np.zeros((sim.N,Nout))
vx = np.zeros((sim.N,Nout))
vy = np.zeros((sim.N,Nout))
vz = np.zeros((sim.N,Nout))

times = np.linspace(0.,tmax,Nout)
ps = sim.particles

for i,time in enumerate(times):
    sim.integrate(time)
    for j in range(sim.N):
        x[j][i] = ps[j].x 
        y[j][i] = ps[j].y
        z[j][i] = ps[j].z
        vx[j][i] = ps[j].vx 
        vy[j][i] = ps[j].vy
        vz[j][i] = ps[j].vz


In [5]:
n = sim.particles[1].n
pi1 = m1/(m1+m2)
pi2 = m2/(m1+m2)
x1 = -pi2*r12
x2 = pi1*r12

In [6]:
thetas = -np.arctan2(y[1],x[1])
rx = x*np.cos(thetas) - y*np.sin(thetas)
ry = x*np.sin(thetas) + y*np.cos(thetas)
rz = z
rvx = vx + y*n 
rvy = vy - x*n 
rvz = vz 

In [7]:
r = np.array([rx[2],ry[2],rz[2]]).T
r1 = np.array([(rx[2]-x1),ry[2],rz[2]]).T
r2 = np.array([(rx[2]-x2),ry[2],rz[2]]).T

In [8]:
centrifugal  = (n*norm(r,axis=1))**2 / 2
gravitational = m1/norm(r1,axis=1) + m2/norm(r2,axis=1) 
kinetic = rvx[2]**2 + rvy[2]**2 + rvz[2]**2
u = centrifugal + gravitational
jacobi=  (2*u - kinetic)
#plt.plot(times,jacobi)

In [9]:
l=2.5
xs = np.linspace(-l,l,1000)
ys = np.linspace(-l,l,1000)

In [10]:
X,Y = np.meshgrid(xs,ys)

In [11]:
R = np.sqrt(X**2 + Y**2)
R1 = np.sqrt((X-x1)**2 + Y**2)
R2 = np.sqrt((X-x2)**2 + Y**2)
U = ((n*R)**2) + 2*(m1/R1 + m2/R2) 

In [12]:
from matplotlib import cm
from animate import *

In [13]:
def makecircle(r):
    t = np.linspace(0,2*np.pi,100)
    return r*np.sin(t), r*np.cos(t)

In [14]:
def makeplot(i):
    fig,ax = plt.subplots(ncols=2,figsize=(20,10),sharex=True,sharey=True)
    plt.subplots_adjust(wspace=0.1)

    umin,umax = np.log10(np.min(U)),0.6*np.log10(np.max(U))
    level= np.logspace(umin,umax,20)
    loglevels = np.log10(level)
    colors = cm.viridis((loglevels - np.min(loglevels)) / (np.max(loglevels) - np.min(loglevels)))

    ax[0].contour(X,Y,U,levels=[jacobi[0]],zorder=-1,colors='k',linewidths=5)
    ax[0].contour(X,Y,U,levels=level,zorder=-1,colors = colors, linewidths=2)
    ax[0].plot(rx[2][:i],ry[2][:i],c='r',alpha=0.3,zorder=1, linewidth=2)
    ax[0].scatter(rx[2][i],ry[2][i],c='r',alpha=1.0,zorder=1,marker='x')
    ax[0].scatter(rx[0][i],ry[0][i],c='k',s=100)
    ax[0].scatter(rx[1][i],ry[1][i],c='r',s=100)
    ax[0].set_xlim(-l,l)
    ax[0].set_ylim(-l,l)
    ax[0].axvline(x1,c='k',ls='--')
    ax[0].axvline(x2,c='r',ls='--')

    ox1,oy1 = makecircle(x1)
    ox2,oy2 = makecircle(x2)
    ax[1].plot(ox1,oy1,ls='--',c='k', linewidth=2)
    ax[1].plot(ox2,oy2,ls='--',c='r', linewidth=2)
    ax[1].scatter(x[0][i],y[0][i],c='k',s=100)
    ax[1].scatter(x[1][i],y[1][i],c='r',s=100)
    ax[1].scatter(x[2][i],y[2][i],c='r',alpha=1.0,zorder=1,marker='x')
    ax[1].plot(x[2][:i],y[2][:i],c='r',alpha=0.3,zorder=1, linewidth=2)
    return fig

In [16]:
savefigures(makeplot,np.arange(0,len(times),20),'./outer_case/',dpival=150)

saving figures


100%|█████████████████████████████████████████| 500/500 [03:20<00:00,  2.49it/s]


In [17]:
render('./outer_case/','outer_case','gif',cleanup_type='rm')

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Framerate is None fps
Total Runtime is None s


[Parsed_palettegen_0 @ 0x55f84fc56d80] 255(+1) colors generated out of 5769 colors; ratio=0.044202
[Parsed_palettegen_0 @ 0x55f84fc56d80] Dupped color: FFFFC2C2
[Parsed_palettegen_0 @ 0x55f84fc56d80] Dupped color: FFFFE4E4
    Last message repeated 1 times
Output #0, image2, to '../temp_palette_outer_case.png':
  Metadata:
    encoder         : Lavf58.76.100
  Stream #0:0: Video: png, rgba(pc, gbr/unknown/unknown, progressive), 16x16 [SAR 1:1 DAR 1:1], q=2-31, 200 kb/s, 25 fps, 25 tbn
    Metadata:
      encoder         : Lavc58.134.100 png
frame=    1 fps=0.2 q=-0.0 Lsize=N/A time=00:00:00.04 bitrate=N/A speed=0.00865x    
video:1kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x